# Multi-task recommenders

In the [basic retrieval tutorial](basic_retrieval) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In addition, building a joint model for multiple tasks may produce better results than building a number of task-specific models. This is especially true where some data is abundant (for example, clicks), and some data is sparse (purchases, returns, manual reviews). In those scenarios, a joint model may be able to use representations learned from the abundant task to improve its predictions on the sparse task via a phenomenon known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). For example, [this paper](https://openreview.net/pdf?id=SJxPVcSonN) shows that a model predicting explicit user ratings from sparse user surveys can be substantially improved by adding an auxiliary task that uses abundant click log data.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [1]:
!ls -al .

total 64
drwx------ 13 root nogroup 6144 Nov  2 21:33 .
drwxr-xr-x  1 root root      39 Nov  2 21:24 ..
-rw-------  1 root root    2230 Nov  2 21:14 .bash_history
drwxr-xr-x  7 root root    6144 Nov  2 20:59 .cache
drwxr-xr-x  3 root root    6144 Nov  2 15:44 .config
-rw-r--r--  1 root root      54 Nov  2 15:39 .gitconfig
drwxr-xr-x  2 root root    6144 Nov  1 19:31 .ipynb_checkpoints
drwxr-xr-x  5 root root    6144 Oct 10 23:13 .ipython
drwxr-xr-x  3 root root    6144 Nov  2 21:24 .jupyter
drwxr-xr-x  2 root root    6144 Nov  1 19:59 .keras
drwxr-xr-x  3 root root    6144 Oct 10 22:41 .local
drwxr-xr-x  2 root root    6144 Nov  1 19:33 .ssh
-rw-r--r--  1 root root     111 Oct 10 22:41 .yarnrc
drwxr-xr-x  3 root root    6144 Nov  2 21:12 exported_models
drwxr-xr-x  3 root root    6144 Nov  2 21:33 model
-rw-r--r--  1 root root     764 Nov  2 21:33 model.tar.gz
drwxr-xr-x 19 root root    6144 Nov  1 19:35 workshop


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're going to use the Movielens 100K dataset.

In [4]:
ls -al .

total 64
drwx------ 13 root nogroup 6144 Nov  2 21:33 ./
drwxr-xr-x  1 root root      39 Nov  2 21:24 ../
-rw-------  1 root root    2230 Nov  2 21:14 .bash_history
drwxr-xr-x  7 root root    6144 Nov  2 20:59 .cache/
drwxr-xr-x  3 root root    6144 Nov  2 15:44 .config/
-rw-r--r--  1 root root      54 Nov  2 15:39 .gitconfig
drwxr-xr-x  2 root root    6144 Nov  1 19:31 .ipynb_checkpoints/
drwxr-xr-x  5 root root    6144 Oct 10 23:13 .ipython/
drwxr-xr-x  3 root root    6144 Nov  2 21:24 .jupyter/
drwxr-xr-x  2 root root    6144 Nov  1 19:59 .keras/
drwxr-xr-x  3 root root    6144 Oct 10 22:41 .local/
drwxr-xr-x  2 root root    6144 Nov  1 19:33 .ssh/
-rw-r--r--  1 root root     111 Oct 10 22:41 .yarnrc
drwxr-xr-x  3 root root    6144 Nov  2 21:12 exported_models/
drwxr-xr-x  3 root root    6144 Nov  2 21:33 model/
-rw-r--r--  1 root root     764 Nov  2 21:33 model.tar.gz
drwxr-xr-x 19 root root    6144 Nov  1 19:35 workshop/


In [5]:
ratings = tfds.load('movielens/100k-ratings',                     
                    download=False,
                    data_dir='/root/workshop/02_usecases/sagemaker_recommendations/tensorflow_datasets/',
                    split='train')

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
print(ratings)

<MapDataset shapes: {movie_title: (), user_id: (), user_rating: ()}, types: {movie_title: tf.string, user_id: tf.string, user_rating: tf.float32}>


In [6]:
movies = tfds.load('movielens/100k-movies',                     
                    download=False,
                    data_dir='/root/workshop/02_usecases/sagemaker_recommendations/tensorflow_datasets/',
                    split='train')

movies = movies.map(lambda x: x["movie_title"])
print(movies)

<MapDataset shapes: (), types: tf.string>


And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [7]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

```python
user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_user_ids),
  # We add 2 to account for unknown and mask tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 2, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_movie_titles),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 2, embedding_dimension)
])
```

However, now we will have two tasks. The first is the rating task:

```python
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)
```

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

```python
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)
```

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [8]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=features["user_rating"],
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [9]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [10]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [11]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"retrieval-top-100-accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"ranking-rmse: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3


10/10 [==============================] - 20s 2s/step - root_mean_squared_error: 2.0903 - factorized_top_k/top_1_categorical_accuracy: 3.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0024 - factorized_top_k/top_10_categorical_accuracy: 0.0055 - factorized_top_k/top_50_categorical_accuracy: 0.0295 - factorized_top_k/top_100_categorical_accuracy: 0.0590 - loss: 4.0315 - regularization_loss: 0.0000e+00 - total_loss: 4.0315
Epoch 2/3
10/10 [==============================] - 21s 2s/step - root_mean_squared_error: 1.1531 - factorized_top_k/top_1_categorical_accuracy: 2.3750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0024 - factorized_top_k/top_10_categorical_accuracy: 0.0054 - factorized_top_k/top_50_categorical_accuracy: 0.0297 - factorized_top_k/top_100_categorical_accuracy: 0.0592 - loss: 1.3189 - regularization_loss: 0.0000e+00 - total_loss: 1.3189
Epoch 3/3
5/5 [==============================] - 3s 579ms/step - root_mean_squared_error: 1.1130 - factorized_top_k/top_

The model does OK on predicting ratings (with an RMSE of around 1.11), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [12]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [13]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"retrieval-top-100-accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"ranking-rmse: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3


10/10 [==============================] - 20s 2s/step - root_mean_squared_error: 3.7237 - factorized_top_k/top_1_categorical_accuracy: 2.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_10_categorical_accuracy: 0.0081 - factorized_top_k/top_50_categorical_accuracy: 0.0696 - factorized_top_k/top_100_categorical_accuracy: 0.1442 - loss: 69818.0284 - regularization_loss: 0.0000e+00 - total_loss: 69818.0284
Epoch 2/3
10/10 [==============================] - 20s 2s/step - root_mean_squared_error: 3.7495 - factorized_top_k/top_1_categorical_accuracy: 0.0016 - factorized_top_k/top_5_categorical_accuracy: 0.0143 - factorized_top_k/top_10_categorical_accuracy: 0.0316 - factorized_top_k/top_50_categorical_accuracy: 0.1542 - factorized_top_k/top_100_categorical_accuracy: 0.2781 - loss: 67473.2876 - regularization_loss: 0.0000e+00 - total_loss: 67473.2876
Epoch 3/3
5/5 [==============================] - 3s 688ms/step - root_mean_squared_error: 3.7730 - factorize

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [14]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [15]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3


10/10 [==============================] - 20s 2s/step - root_mean_squared_error: 2.5007 - factorized_top_k/top_1_categorical_accuracy: 2.1250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0076 - factorized_top_k/top_50_categorical_accuracy: 0.0640 - factorized_top_k/top_100_categorical_accuracy: 0.1391 - loss: 69811.8246 - regularization_loss: 0.0000e+00 - total_loss: 69811.8246
Epoch 2/3
10/10 [==============================] - 20s 2s/step - root_mean_squared_error: 1.2097 - factorized_top_k/top_1_categorical_accuracy: 0.0015 - factorized_top_k/top_5_categorical_accuracy: 0.0137 - factorized_top_k/top_10_categorical_accuracy: 0.0298 - factorized_top_k/top_50_categorical_accuracy: 0.1488 - factorized_top_k/top_100_categorical_accuracy: 0.2721 - loss: 67481.2720 - regularization_loss: 0.0000e+00 - total_loss: 67481.2720
Epoch 3/3
5/5 [==============================] - 3s 692ms/step - root_mean_squared_error: 1.1311 - factorize

The result is a model that performs roughly as well on both tasks as each specialized model. 

While the results here do not show a clear accuracy benefit from a joint model in this case, multi-task learning is in general an extremely useful tool. We can expect better results when we can transfer knowledge from a data-abundant task (such as clicks) to a closely related data-sparse task (such as purchases).

In [16]:
# Create a model that takes in raw query features, and returns the predicted movie titles
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

In [17]:
k = 5
user_id = "42"

_, titles = index(np.array([user_id]))

print(f"Top {k} recommendations for user {user_id}: {titles[0, :k]}")

Top 5 recommendations for user 42: [b'Homeward Bound: The Incredible Journey (1993)' b'Jack (1996)'
 b'First Wives Club, The (1996)' b'101 Dalmatians (1996)'
 b'Rent-a-Kid (1995)']


In [18]:
print('Trained index {}'.format(index))          
print(index.summary())

Trained index <tensorflow_recommenders.layers.factorized_top_k.BruteForce object at 0x7fba1204b3d0>
Model: "brute_force"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_7 (Sequential)    (None, 32)                30208     
Total params: 85,714
Trainable params: 30,208
Non-trainable params: 55,506
_________________________________________________________________
None


In [19]:
# model.retrieval_task = tfrs.tasks.Retrieval()  # Removes the metrics.
# model.compile()

In [20]:
# print('Compiled model {}'.format(model))          
# print(model.summary())

In [21]:
import os

local_model_dir_multitask_model = './exported_models/multitask_model/'

tensorflow_saved_model_path_multitask_model = os.path.join(local_model_dir_multitask_model, 'tensorflow/saved_model/0')

os.makedirs(tensorflow_saved_model_path_multitask_model, exist_ok=True)

In [22]:
index.save(tensorflow_saved_model_path_multitask_model, save_format='tf')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ./exported_models/multitask_model/tensorflow/saved_model/0/assets


INFO:tensorflow:Assets written to: ./exported_models/multitask_model/tensorflow/saved_model/0/assets


In [23]:
!saved_model_cli show --all --dir $tensorflow_saved_model_path_multitask_model


2020-11-02 21:39:37.873096: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-02 21:39:37.873147: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_in

In [25]:
!saved_model_cli run --input_exprs 'input_1=["$user_id"]' --tag_set serve --signature_def serving_default --dir $tensorflow_saved_model_path_multitask_model

2020-11-02 21:41:59.531308: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-02 21:41:59.531358: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-11-02 21:42:00.803111: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-11-02 21:42:00.803152: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2020-11-02 21:42:00.803186: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395): /proc/driver/nvidia/version does not exist
2020-11-02 21:42:00